In [1]:
import os
import pandas as pd
import requests
import pathlib
from requests.auth import HTTPBasicAuth

api_key = os.environ.get("NCBI_API_KEY")

In [2]:
# Input
taxon_file = "../data/imported/taxids.list"

# Output
output_dir = pathlib.Path("../data/generated/download_genomes")
pathlib.Path.mkdir(output_dir, exist_ok=True)
genomes_metadata_file = output_dir / "genomes_metadata.csv"
genome_accessions_file = output_dir / "genome_accessions.list"
genomes_zip_file = output_dir / "genomes.zip"

In [3]:
def get_genome_dataset_report(taxon_id, api_key, params=None):
    """
    Get the genome dataset report for a given taxon id using NCBI API.
    Default parameters are:
    - reference_only=true
    - assembly_source=refseq
    - exclude_atypical=true # => don't include genomes that have assembly issues or are otherwise atypical
    - assembly_version=current
    """
    if params is None:
        params = {
            "filters.reference_only": "true",
            "filters.assembly_source": "refseq",
            "filters.exclude_atypical": "true",
            "filters.assembly_version": "current",
        }
    auth = HTTPBasicAuth("api-key", api_key)
    url = f"https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/taxon/{taxon_id}/dataset_report"
    response = requests.get(url=url, auth=auth, params=params)
    if "reports" not in response.json():
        print(f"No reports found for taxon id {taxon_id}")
        return
    return pd.DataFrame(
        _parse_genome_dataset_report_response(response.json(), taxon_id)
    )


def _parse_genome_dataset_report_response(response, query_taxid):
    for report in response["reports"]:
        yield {
            "query_taxid": query_taxid,
            "genome_accession": report["current_accession"],
            "genome_name": report["organism"]["organism_name"],
            "genome_taxid": report["organism"]["tax_id"],
            "assembly_level": report["assembly_info"]["assembly_level"],
            "assembly_status": report["assembly_info"]["assembly_status"],
            "assembly_name": report["assembly_info"]["assembly_name"],
            "refseq_category": report["assembly_info"]["refseq_category"]
            if "refseq_category" in report["assembly_info"]
            else pd.NA,
            "number_of_contigs": report["assembly_stats"]["number_of_contigs"],
            "contig_n50": report["assembly_stats"]["contig_n50"],
            "contig_l50": report["assembly_stats"]["contig_l50"],
            "genome_coverage": float(
                str(report["assembly_stats"]["genome_coverage"]).replace("x", "")
            )
            if "genome_coverage" in report["assembly_stats"]
            else pd.NA,
            "total_sequence_length": report["assembly_stats"]["total_sequence_length"],
            "total_ungapped_length": report["assembly_stats"]["total_ungapped_length"],
        }

In [4]:
refseq_params = {
    "filters.reference_only": "true",
    "filters.assembly_source": "refseq",
    "filters.exclude_atypical": "true",
    "filters.assembly_version": "current",
}

genome_dataset_report = get_genome_dataset_report("562", api_key, params=refseq_params)
genome_dataset_report

,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
0,562,GCF_000005845.2,Escherichia coli str. K-12 substr. MG1655,511145,Complete Genome,current,ASM584v2,reference genome,1,4641652,1,<NA>,4641652,4641652
1,562,GCF_000008865.2,Escherichia coli O157:H7 str. Sakai,386585,Complete Genome,current,ASM886v2,reference genome,3,5498578,1,<NA>,5594605,5594605


In [5]:
def read_taxon_list(taxon_file):
    """
    Read a list of taxon ids from a file.
    """
    taxon_list = []
    with open(taxon_file) as f:
        taxon_list.extend(line.strip() for line in f if not line.startswith("#"))
    return sorted(set(taxon_list))


taxon_list = read_taxon_list(taxon_file)
print(f"{len(taxon_list)} taxon ids read from {taxon_file}")

161 taxon ids read from ../data/imported/taxids.list


In [6]:
refseq_genomes_dataset_reports = [
    get_genome_dataset_report(taxon_id, api_key, params=refseq_params)
    for taxon_id in taxon_list
]
refseq_genomes_df = pd.concat(refseq_genomes_dataset_reports).reset_index(drop=True)
refseq_genomes_df.sample()

No reports found for taxon id 1235797
No reports found for taxon id 142586
No reports found for taxon id 1454229
No reports found for taxon id 1686296
No reports found for taxon id 1849041
No reports found for taxon id 1935934
No reports found for taxon id 1955243
No reports found for taxon id 1965293
No reports found for taxon id 2039241
No reports found for taxon id 2040332
No reports found for taxon id 2049021
No reports found for taxon id 39491
No reports found for taxon id 39492
No reports found for taxon id 41978
No reports found for taxon id 885


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
128,61171,GCF_003459085.1,Holdemania filiformis,61171,Scaffold,current,ASM345908v1,representative genome,58,141100,9,100.0,3744856,3744494


In [7]:
refseq_genomes_df["refseq_category"].value_counts()

representative genome    146
Name: refseq_category, dtype: int64

In [8]:
refseq_genomes_df["assembly_level"].value_counts()

Complete Genome    70
Scaffold           42
Contig             25
Chromosome          9
Name: assembly_level, dtype: int64

In [9]:
failed_taxon_ids = list(
    set(taxon_list).difference(set(refseq_genomes_df["query_taxid"]))
)
print(f"{len(failed_taxon_ids)} taxon ids failed to retrieve genome dataset report")
failed_taxon_ids

15 taxon ids failed to retrieve genome dataset report


['2040332',
 '1955243',
 '1235797',
 '1686296',
 '1849041',
 '142586',
 '885',
 '1935934',
 '39491',
 '1965293',
 '39492',
 '2039241',
 '41978',
 '2049021',
 '1454229']

In [10]:
genbank_params = {
    "filters.assembly_source": "genbank",
    "filters.exclude_atypical": "true",
    "filters.assembly_version": "current",
    "table_fields": "assminfo-accession,assminfo-name",
    "page_size": 500,
}

genbank_genome_dataset_reports = [
    get_genome_dataset_report(failed_taxon_id, api_key, params=genbank_params)
    for failed_taxon_id in failed_taxon_ids
]
genbank_genomes_df = pd.concat(genbank_genome_dataset_reports).reset_index(drop=True)
genbank_genomes_df.sample(5)

No reports found for taxon id 1849041


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
849,39492,GCA_000209915.1,[Eubacterium] siraeum 70/3,657319,Chromosome,current,ASM20991v1,<NA>,139,38521,22,<NA>,2943413,2864137
643,39491,GCA_003469095.1,[Eubacterium] rectale,39491,Scaffold,current,ASM346909v1,<NA>,135,122438,8,100.0,3243530,3242497
833,39491,GCA_934871245.1,[Eubacterium] rectale,39491,Scaffold,current,MTG238_bin.4.fa,<NA>,65,88437,9,20.0,2738878,2738178
43,2040332,GCA_025541975.1,Dorea sp.,2040332,Contig,current,ASM2554197v1,<NA>,47,128628,5,2.6,2259685,2259685
436,142586,GCA_022800015.1,Eubacterium sp.,142586,Contig,current,ASM2280001v1,<NA>,149,30263,28,73.0,2954030,2954030


In [11]:
genbank_genomes_df["assembly_level"].value_counts()

Contig             973
Scaffold           481
Chromosome           8
Complete Genome      3
Name: assembly_level, dtype: int64

In [12]:
def select_best_representative(df, consider_coverage=None):
    """
    Select the best representative genome from a genome dataset report.
    """
    if consider_coverage is None:
        consider_coverage = True
    taxa_dfs = []
    # convert columns to correct data types
    df = df.astype(
        {
            "query_taxid": "int",
            "genome_taxid": "int",
            "number_of_contigs": "int",
            "total_sequence_length": "int",
            "total_ungapped_length": "int",
            "contig_n50": "int",
            "contig_l50": "int",
            # "genome_coverage": "float",
        }
    )
    df["gaps"] = df["total_sequence_length"] - df["total_ungapped_length"]
    df["bases_per_contig"] = df["total_ungapped_length"] / df["number_of_contigs"]

    if consider_coverage:
        # if genome coverage is NA remove it from the dataframe
        df = df.dropna(subset=["genome_coverage"]).copy()

    # Select the representative genome with the highest genome coverage, least gaps and least contig_l50 and highest contig_n50.
    for _, grouped_taxa_df in df.groupby("query_taxid"):
        rows = grouped_taxa_df.shape[0]
        if rows == 1:
            taxa_dfs.append(grouped_taxa_df.iloc[0])
            continue

        # if a taxon has any 'Complete Genome's then shortlist those genomes to be considered for a representative genome.
        if any(grouped_taxa_df["assembly_level"] == "Complete Genome"):
            grouped_taxa_df = grouped_taxa_df.query(
                "assembly_level == 'Complete Genome'"
            ).copy()

        # Select the representative genome with the highest genome coverage, least gaps and least contig_l50 and highest contig_n50.
        grouped_taxa_df = grouped_taxa_df.sort_values(
            ["bases_per_contig", "contig_n50", "contig_l50", "genome_coverage", "gaps"],
            ascending=[False, False, True, False, True],
        )

        # append the best representative genome to the taxa_dfs list
        taxa_dfs.append(grouped_taxa_df.iloc[0])

    return pd.DataFrame(taxa_dfs)


select_best_representative(genbank_genomes_df).reset_index(drop=True).sample()

,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length,gaps,bases_per_contig
5,1235797,GCA_000403435.2,Oscillibacter sp. 1-3,1235797,Scaffold,current,Osci_bact_1-3_V1,<NA>,43,271557,6,88.0,4467686,4430205,37481,103028.023256


In [13]:
genomes_df = pd.concat(
    [
        select_best_representative(refseq_genomes_df, consider_coverage=False),
        select_best_representative(genbank_genomes_df),
    ]
).reset_index(drop=True)
genomes_df.to_csv(genomes_metadata_file, index=False)
genomes_df.sample(5)

,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length,gaps,bases_per_contig
113,446660,GCF_000478885.1,Adlercreutzia equolifaciens DSM 19450,1384484,Complete Genome,current,ASM47888v1,representative genome,1,2862526,1,33.1,2862526,2862526,0,2.862526e+06
64,46506,GCF_000154525.1,Bacteroides stercoris ATCC 43183,449673,Scaffold,current,ASM15452v1,representative genome,23,587129,3,<NA>,4009829,4009229,600,1.743143e+05
48,33038,GCF_009831375.1,[Ruminococcus] gnavus ATCC 29149,411470,Complete Genome,current,ASM983137v1,representative genome,1,3549191,1,150.0,3549191,3549191,0,3.549191e+06
22,1547,GCF_016728785.1,Thomasclavelia ramosa,1547,Complete Genome,current,ASM1672878v1,representative genome,1,3543719,1,751.0,3543719,3543719,0,3.543719e+06
7,907,GCF_003006415.1,Megasphaera elsdenii,907,Complete Genome,current,ASM300641v1,representative genome,1,2566193,1,137.0,2566193,2566193,0,2.566193e+06


In [16]:
genomes_df["genome_accession"].value_counts()

GCF_016889925.1    1
GCF_014131755.1    1
GCF_020341675.1    1
GCF_025149125.1    1
GCF_008571395.1    1
                  ..
GCF_000169235.1    1
GCF_018140635.1    1
GCF_003475105.1    1
GCF_902375065.1    1
GCA_028727085.1    1
Name: genome_accession, Length: 160, dtype: int64

In [14]:
## Check to make sure there are no duplicate genome accessions
assert all(
    genomes_df["genome_accession"].value_counts() == 1
), "Duplicate genome accessions found!"

In [15]:
# Save the genome accessions to a file
genomes_df["genome_accession"].to_csv(genome_accessions_file, index=False, header=False)

## Download the genomes

In [16]:
%%bash -s "$api_key" "$genome_accessions_file" "$genomes_zip_file"

datasets download genome accession \
    --api-key $1 \
    --inputfile $2 \
    --no-progressbar \
    --filename $3
    